In [ ]:
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.data import MetadataCatalog, DatasetCatalog, catalog
from detectron2.structures import BoxMode
from os import listdir
from os.path import isfile, join
from detectron2.data.datasets import register_coco_instances
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np
import shutil
import os

In [ ]:
background_image_folder = './data/classifier/background_images'
product_image_folder = './data/classifier/images_18skus'
combined_output_folder = './data/classifier/combined'
video_folder = './data/cps-test-videos'

In [ ]:
def generate_background_images():
    if os.path.exists(background_image_folder):
        shutil.rmtree(background_image_folder)
    os.makedirs(background_image_folder)
    total_count = 0
    for video_name in listdir(video_folder):
        video_path = join(video_folder, video_name)
        capture = cv2.VideoCapture(video_path)
        idx = 0
        while True:
            ret, frame_img = capture.read()
            if frame_img is None:
                break
            if idx % 6 == 5:
                output_name = join(background_image_folder, video_name + '_' + str(idx) + '.jpg')
                cv2.imwrite(output_name,frame_img)
                total_count +=1
            idx+=1
            
    print ('number of background images', total_count)
    
generate_background_images()

In [ ]:
def generate_train_image(background_image_path, product_image_path, combined_output_path, product_id, category_id, image_id):         
    pdt = cv2.imread(product_image_path)
    # TODO: blur boundary
    # TODO: black in product
    mask = np.logical_and(pdt[:,:,0]>0, pdt[:,:,1]>0, pdt[:,:,2]>0)
    mask = np.dstack([mask]*3)
    bg = cv2.imread(background_image_path)
    (pdt_y, pdt_x, _) = pdt.shape
    (bg_y, bg_x, _) = bg.shape

    combined = bg
    x_offset = random.randint(0,bg_x - pdt_x)
    y_offset = random.randint(0,bg_y - pdt_y)
    bg_cropped = combined[y_offset:y_offset+pdt_y, x_offset:x_offset+pdt_x]

    np.putmask(bg_cropped, mask, pdt)
    combined[y_offset:y_offset+pdt_y, x_offset:x_offset+pdt_x] = bg_cropped
#     cv2.rectangle(combined,(x_offset,y_offset),(x_offset+pdt_x,y_offset+pdt_y),(0,0,255),3)
    record = {}
        
    record["file_name"] = combined_output_path
    record["image_id"] = image_id
    record["height"] = bg_y
    record["width"] = bg_x

    boxes = [x_offset, y_offset, x_offset+pdt_x, y_offset+pdt_y]
    objs = []
    obj = {
        "bbox": boxes,
        "bbox_mode": BoxMode.XYXY_ABS,
        #"segmentation": [poly], To draw a line, along to ballon
        #you will need this for mask RCNN
        "category_id": category_id,
        "iscrowd": 0
    }
    objs.append(obj)
    record["annotations"] = objs
    cv2.imwrite(combined_output_path, combined)
    return combined, record

# background_image_folder
#   |_ 1.jpg
#   |_ 2.jpg
# product_image_folder
#   |_product1_id
#       |_ 1.jpg
#       |_ 2.jpg
#   |_product2_id
#       |_ 1.jpg
#       |_ 2.jpg
def generate_train_images():
    # get background images
    bg_imgs = [join(background_image_folder, f) for f in listdir(background_image_folder) if f.endswith('.png') or f.endswith('.jpg')]
    bg_imgs_count = len(bg_imgs)
    print ('number of background images', bg_imgs_count)
    dataset_dicts = []
    
    if os.path.exists(combined_output_folder):
        shutil.rmtree(combined_output_folder)
    os.makedirs(combined_output_folder)
    
    for product_id in listdir(product_image_folder):
        in_folder = join(product_image_folder, product_id) 
        out_folder = join(combined_output_folder, product_id) 
        if isfile(in_folder):
            continue
        os.makedirs(out_folder)
        category_id = len(product_barcodes)
        product_barcodes.append(product_id)
        
        count = 0
        for f in listdir(in_folder):
            if not f.endswith('.png') and not f.endswith('.jpg'):
                continue
            count += 1
            if count % 10 != 0:
                continue
            product_image_path = join(in_folder, f)
            background_image_path = bg_imgs[random.randint(0, bg_imgs_count-1)]
            image_id = product_id + '_' + str(count)
            combined_output_path = join(combined_output_folder, product_id, image_id+ '.jpg')
            
            combined, record = generate_train_image(background_image_path, product_image_path, combined_output_path, product_id, category_id, image_id)
            
            dataset_dicts.append(record)
#             combined = cv2.cvtColor(combined, cv2.COLOR_BGR2RGB)
#             fig = plt.figure(figsize=(20,10))
#             plt.imshow(combined)
    print ('number of products', len(product_barcodes))
    print ('number of product images', len(dataset_dicts))
    print ('number of generated combined images', len(dataset_dicts))
    return dataset_dicts 

product_barcodes = []
DatasetCatalog._REGISTERED.clear()
MetadataCatalog._NAME_TO_META.clear()
DatasetCatalog.register('test', generate_train_images)
MetadataCatalog.get('test').set(thing_classes=product_barcodes)
convert_to_coco_json('test', output_file='./data/classifier/combined/metadata_coco_format.json', allow_cached=False)

In [ ]:
# TODO: product image size too big comparing to bg?
# TODO: boundary smoothing?